In [1]:
import duckdb

In [2]:
# สร้าง DuckDB และเชื่อมต่อ
con = duckdb.connect('ecommerce.duckdb')

In [3]:
# โหลด historical data
con.execute("""
    CREATE OR REPLACE TABLE historical AS
    SELECT * FROM read_parquet('historical_transactions.parquet')
""")

In [4]:
# โหลด daily data ทั้ง 5 วัน
for day in range(1, 6):
    con.execute(f"""
        INSERT INTO historical
        SELECT * FROM read_parquet('daily_transactions_day{day}.parquet')
    """)

In [5]:
# 1. The number of void/cancel transactions without an initial payment.
query_void_without_completed = """
SELECT COUNT(*) AS void_or_cancel_without_completed
FROM historical h
WHERE status IN ('void', 'cancel')
  AND NOT EXISTS (
    SELECT 1 FROM historical h2
    WHERE h2.transaction_id = h.transaction_id
      AND h2.status = 'completed'
  )
"""

print(con.execute(query_void_without_completed).fetchdf())

   void_or_cancel_without_completed
0                              5000


In [6]:
# 2. The count of negative transaction amounts.
query_negative_amounts = """
SELECT COUNT(*) AS negative_amounts
FROM historical
WHERE amount < 0
"""

print(con.execute(query_negative_amounts).fetchdf())

   negative_amounts
0              2100


In [7]:
# 3. The distribution of transaction statuses.
query_status_distribution = """
SELECT status, COUNT(*) AS count
FROM historical
GROUP BY status
"""

print(con.execute(query_status_distribution).fetchdf())

      status    count
0     cancel     2666
1       void     2584
2  completed  1044750


In [8]:
# Generate summary statistics on transaction trends.
query_trends = """
SELECT SUBSTR(timestamp, 1, 10) AS date, COUNT(*) AS total_transactions
FROM historical
GROUP BY date
ORDER BY date
"""

print(con.execute(query_trends).fetchdf())

           date  total_transactions
0    2024-04-22                1478
1    2024-04-23                2763
2    2024-04-24                2743
3    2024-04-25                2718
4    2024-04-26                2710
..          ...                 ...
365  2025-04-22               11965
366  2025-04-23               10000
367  2025-04-24               10000
368  2025-04-25               10000
369  2025-04-26               10000

[370 rows x 2 columns]


In [10]:
con.close()